<a href="https://colab.research.google.com/github/Dheeraj13082004/Celiac_Disease_Prediction/blob/main/Celiac_Disease_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'celiac-disease-coeliac-disease:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3025095%2F5226711%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240422%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240422T105446Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D49573a232369e3b045571a8d209bef78a0c8515dc4373abeabe511f1afa12f6aa09e12a1653e717b28703f2e8d6e3a18745ee8b10f5edc85256a4a62fd88768c0a2028e62acb44e2123b17ba9dcce1f420d7f067d31617acfcddc10e391f095ebd894b917bf7e17da016d974688500f88c9c6c821870d8499aa0ef0440d393d8f08b3063fdb8b21cf089c3dc552977c72be963d4f6601d7fe1ad678c9f71fe5bbc06e8c89ed607ff7e80d7cef550d8d7090da1fe352b4427fe37454c29927116484e25af6418e0adf7775ff2927e72d2668d1b565c3bdc98404baad2f6b719afa1d3d1c5986338efe805f1e917088fde202a47556d15978ef2d4548bbc5cad8e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## Working on data

In [ ]:
df=pd.read_csv("/kaggle/input/celiac-disease-coeliac-disease/celiac_disease_lab_data.csv")
print(3*"\n",40*"=","First 15 instances",40*"=")
display(df.head(15))
print(3*"\n",40*"=","Information about data",40*"=")
display(df.info())
print(3*"\n",40*"=","Data set shape",40*"=")
print(df.shape)

## Handling Duplicated Data

In [ ]:
print(df.duplicated().sum())

In [ ]:
df.drop_duplicates(inplace=True)
print(df.shape)

## Handling Null Values

In [ ]:
print(df.isnull().sum())

**If Diabetes answer is no so there is no type for diabetes so will assign value type 0**

In [ ]:
df["Diabetes Type"].fillna("Type 0",inplace=True)
display(df.head(15))

## Data encoding

In [ ]:
print(df.dtypes)

In [ ]:
encode=LabelEncoder()
df["Gender"]=encode.fit_transform(df["Gender"])
df["Diabetes"]=encode.fit_transform(df["Diabetes"])
df["Diabetes Type"]=encode.fit_transform(df["Diabetes Type"])
df["Diarrhoea"]=encode.fit_transform(df["Diarrhoea"])
df["Abdominal"]=encode.fit_transform(df["Abdominal"])
df["Short_Stature"]=encode.fit_transform(df["Short_Stature"])
df["Sticky_Stool"]=encode.fit_transform(df["Sticky_Stool"])
df["Weight_loss"]=encode.fit_transform(df["Weight_loss"])
df["Marsh"]=encode.fit_transform(df["Marsh"])
df["cd_type"]=encode.fit_transform(df["cd_type"])
df["Disease_Diagnose"]=encode.fit_transform(df["Disease_Diagnose"])
print(df.dtypes)

## Data Visualization

**Show distribution of data**

In [ ]:
sns.pairplot(df)
plt.show()

**HeatMap to show correlation**

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(df.corr(),annot=True)
plt.show()

## Working on Features

In [ ]:
Features=df.iloc[:,:-1]
Target=df.iloc[:,-1]

- Drop Diabetes becauese if answer yes Diabetes type will be Type 1 or Type 2
  if answer was no Diabetes type will be Type 0 so we can replace Diabetes with Diabetes type
  
- Drop  Diarrhoea ,IgA ,IgG ,and IgM based on heatmap that show correlation

In [ ]:
Features.drop(["Diabetes","Diarrhoea","IgA","IgG","IgM"],axis=1,inplace=True)

In [ ]:
Features_train,Features_test,Target_train,Target_test=train_test_split(Features,Target,train_size=0.75,random_state=21)

## Training Model

In [ ]:
Model=RandomForestClassifier(random_state=5,n_estimators=11)
Model.fit(Features_train,Target_train)

In [ ]:
print(f"Accurcy of training = {Model.score(Features_train,Target_train)*100:.2f}\nAccurcy of testing = {Model.score(Features_test,Target_test)*100:.2f}")

In [ ]:
Prediction=Model.predict(Features)
score=accuracy_score(Target,Prediction)
print(f"Accurucy of our model = {score*100:.2f}")